# LLM-Based Eng-to-Prsn Translation System Using In-Context Learning

1. Model Choice (Light LLaMA-Based)

Recommended lightweight LLaMA-family models on Hugging Face:

✅ TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0
✅ NousResearch/Llama-2-7b-chat-hf (if you have GPU ≥16GB)

For a light notebook, we’ll use:

## Install dependencies

In [3]:
!pip install transformers accelerate sentencepiece sacrebleu datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.8 MB/s eta 0:00:00


In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

## Import libraries

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import sacrebleu

## Load the Pretrained LLaMA Model

In [30]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-6953cfa0-3f8edb466e19ef01001c5561;6f396ec5-47c8-45cc-a499-996aaa9c027d)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

## Dataset (English → Persian)


*   opus100 (en–fa)

*   Tatoeba

*   Custom CSV

In [6]:
dataset = load_dataset("opus100", "en-fa", split="test[:100]")

README.md: 0.00B [00:00, ?B/s]

en-fa/test-00000-of-00001.parquet:   0%|          | 0.00/193k [00:00<?, ?B/s]

en-fa/train-00000-of-00001.parquet:   0%|          | 0.00/82.4M [00:00<?, ?B/s]

en-fa/validation-00000-of-00001.parquet:   0%|          | 0.00/190k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
len(dataset)

100

In [9]:
dataset[0]["translation"]


{'en': "I'm obsolete!", 'fa': 'من دیگه به دردی نمیخورم'}

## In-Context Learning Prompt Design

Prompt Template

few-shot examples are provided inside the prompt.

In [10]:
def build_prompt(examples, source_sentence):
    prompt = "Translate the following sentences from English to Persian.\n\n"

    for ex in examples:
        prompt += f"English: {ex['en']}\n"
        prompt += f"Persian: {ex['fa']}\n\n"

    prompt += f"English: {source_sentence}\nPersian:"
    return prompt

In [17]:
n = 5
icl_examples = dataset[0:n]["translation"]

In [18]:
examples

[{'en': "I'm obsolete!", 'fa': 'من دیگه به دردی نمیخورم'},
 {'en': "You've really gotta help me now.",
  'fa': 'تو واقعا ديگه بايد کمکم کني.'},
 {'en': 'The entire time you were in rehab,',
  'fa': '\u200fکل مدت\u200cزمانی که توی بازپروری بودی'},
 {'en': 'How well do you know this Dante ?',
  'fa': 'چقدر اين دانته رو ميشناسي؟'},
 {'en': 'I left my other clothes inside.', 'fa': 'لباس قبلي هامو جا گذاشتم'}]

## Translation Function

In [28]:
def translate(sentence, examples):
    prompt = build_prompt(examples, sentence)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("Persian:")[-1].strip()


In [21]:
from tqdm import tqdm

In [22]:
references = []
hypotheses = []

for item in tqdm(dataset[n:]["translation"], desc="Translating"):
    en = item["en"]
    fa = item["fa"]

    pred = translate(en, icl_examples)

    references.append([fa])
    hypotheses.append(pred)

Translating: 100%|██████████| 95/95 [05:00<00:00,  3.17s/it]


In [27]:
print(f"{references[1]} \n {hypotheses[1]}")

['"از هرکسي، با توجه به قابليتش'] 
 من نه روبوت.

English: I'm


## BLEU Score Evaluation

In [23]:
bleu = sacrebleu.corpus_bleu(hypotheses, references)
print(f"BLEU score: {bleu.score:.2f}")

BLEU score: 0.00
